# PyTorch IPC (Inter-Process Communication) Examples

This notebook demonstrates how to share memory between processes using PyTorch.

**Note:** True multiprocessing with `spawn` doesn't work directly in Jupyter notebooks because child processes can't pickle the function definitions. We'll show:
1. A threading example that works in the notebook
2. How to run the full multiprocessing examples from a Python script

## Option 1: Threading (Works in Notebook)

Threads share memory naturally in Python, making this approach simpler for notebook environments:

In [5]:
import torch
import threading

def thread_worker(shared_tensor, worker_id):
    """Thread worker that modifies shared tensor"""
    print(f"Thread {worker_id} starting...")
    shared_tensor[worker_id] = worker_id * 10
    print(f"Thread {worker_id} set position {worker_id} to {shared_tensor[worker_id].item()}")

# Create shared tensor (threads share memory naturally)
shared_tensor = torch.zeros(5)
print("Initial tensor:", shared_tensor)

# Create and start threads
threads = []
for i in range(5):
    t = threading.Thread(target=thread_worker, args=(shared_tensor, i))
    t.start()
    threads.append(t)

# Wait for all threads
for t in threads:
    t.join()

print("\nFinal tensor:", shared_tensor)
print("All threads completed!")

Initial tensor: tensor([0., 0., 0., 0., 0.])
Thread 0 starting...
Thread 0 set position 0 to 0.0
Thread 1 starting...
Thread 2 starting...
Thread 2 set position 2 to 20.0
Thread 1 set position 1 to 10.0
Thread 3 starting...
Thread 4 starting...
Thread 4 set position 4 to 40.0
Thread 3 set position 3 to 30.0

Final tensor: tensor([ 0., 10., 20., 30., 40.])
All threads completed!


## Option 2: True Multiprocessing (Run from Script)

For true process isolation and to fully demonstrate `torch.multiprocessing` with shared memory, run the script:

In [6]:
%%bash
cd /mnt/d/packing/code/analysis
~/miniconda3/envs/rapids-25.10/bin/python ipc_demo.py


PyTorch IPC Examples
Using multiprocessing to demonstrate shared memory

EXAMPLE 1: Basic Shared Memory
Initial tensor: tensor([0., 0., 0., 0., 0.])
Worker 0 starting...
Worker 4 starting...
Worker 2 starting...
Worker 3 starting...
Worker 1 starting...
Worker 0 set position 0 to 0.0
Worker 4 set position 4 to 40.0
Worker 2 set position 2 to 20.0
Worker 3 set position 3 to 30.0
Worker 1 set position 1 to 10.0
Final tensor: tensor([ 0., 10., 20., 30., 40.])
All workers completed!

EXAMPLE 2: Producer-Consumer Pattern
Producer: Generating data...
Consumer: Waiting for data...
Consumer: Read position 0 = 0.0
Consumer: Read position 1 = 1.0
Consumer: Read position 2 = 4.0
Consumer: Read position 3 = 9.0
Consumer: Read position 4 = 16.0
Consumer: Read position 5 = 25.0
Consumer: Read position 6 = 36.0
Consumer: Read position 7 = 49.0
Consumer: Read position 8 = 64.0
Consumer: Read position 9 = 81.0
Producer: Done
Consumer: Done

Final shared data: tensor([ 0.,  1.,  4.,  9., 16., 25., 36.,

The script demonstrates three patterns:

1. **Basic Shared Memory**: 5 workers each write to different positions
2. **Producer-Consumer**: One process produces data, another consumes it with queue synchronization
3. **Parallel Computation**: 4 workers process different slices of a 1000-element tensor

### Key Concepts:
- Use `.share_memory_()` to make tensors shareable across processes
- Use `torch.multiprocessing` instead of regular `multiprocessing`
- Shared tensors must be in CPU memory (each process copies to/from GPU)
- Use `mp.Queue()` for synchronization between processes

### View the multiprocessing script source:

In [7]:
with open('ipc_demo.py', 'r') as f:
    print(f.read())

#!/usr/bin/env python3
"""
PyTorch IPC (Inter-Process Communication) Examples
Demonstrates shared memory between processes using torch.multiprocessing
"""

import torch
import torch.multiprocessing as mp
import time

# Example 1: Basic shared memory tensor
def worker(shared_tensor, worker_id):
    """Worker process that modifies shared tensor"""
    print(f"Worker {worker_id} starting...")
    
    # Each worker adds its ID to its assigned position
    shared_tensor[worker_id] = worker_id * 10
    
    # Simulate some work
    time.sleep(0.1)
    
    print(f"Worker {worker_id} set position {worker_id} to {shared_tensor[worker_id].item()}")


def example1_basic_shared_memory():
    """Basic example of shared memory between processes"""
    print("\n" + "="*60)
    print("EXAMPLE 1: Basic Shared Memory")
    print("="*60)
    
    # Create a tensor and move it to shared memory
    shared_tensor = torch.zeros(5)
    shared_tensor.share_memory_()  # Make tensor shareable across processes
